# Extracting the Dragonfly FOV from the EAGLE simulations

This script is a snippet to extract an array the size of the Dragonfly FOV from the EAGLE data (assumes that the full size 100Mpc data array is input to it).  The shape of the data is not necessary 32000x32000 anymore, because in the load_data step it may have decreased the size/resolution of the data.

The Dragonfly FOV depends on the distance and resolution/size of the data (because the script needs to know the pixel scale).  Give it a reference point that it will center the Dragonfly FOV on.

In [1]:
import numpy as np

In [2]:
def extractFOV(data_tuple, resolution, distance, x_center = 47.5, y_center = 12.,debug=False,x_angFOV=10800.,y_angFOV=7200.):
    """
    data_tuple: [data, newsize, factor]
    data:  2D array, equal to 100Mpc along each size
    newsize: the cropped size of the 2D array before it was binned with imreduce 
    factor:  the factor by which the size of the array was decreased from 32000x32000 i.e. the factor of binned pixels
    resolution: the new resolution of the data, in arcsec (redundant variable, no longer used, since assuming that size
                of the data is 100Mpc a side -- change this so that can input the size of the data)
                resolution = ( size angular ) / ( size pixels ) = ( sizeMpc / pixscale ) / sizepixels
                e.g. resolution = ( 100Mpc / pixscale['100Mpc'] ) / (32000/2) 
    distance: the distance that the data is put at (required for correct scaling wrt cMpc/arcsec)
    x_center, x_center: set of coordinates in Mpc that the output array will be centered on
    Dragonfly FOV is the default desired output angular size:
    x_angFOV: 3.*60.*60. = 10800.0 # arcsec
    y_angFOV: 2.*60.*60. = 7200.0# arcsec
    """
    
    # Using comoving distance to calculate the pixel scale, because the EAGLE simulation is 
    # 100 comoving Mpc a side (so this is consistent).
    pixscale =  {'20Mpc': 0.098/1000.*(1.+0.0047), '25Mpc': 0.120/1000.*(1.+0.0058), \
                 '30Mpc': 0.145/1000.*(1.+0.007), '35Mpc': 0.170/1000.*(1.+0.0082), \
                 '40Mpc': 0.194/1000.*(1.+0.0094),  \
                 '46Mpc': 0.221/1000.*(1.+0.0107), '50Mpc': 0.237/1000.*(1.+0.0115), \
                 '92Mpc': 0.437/1000.*(1.+0.0215),'100Mpc': 0.477/1000.*(1.+0.0235),\
                 '200Mpc': 0.928/1000.*(1.+0.047), '500Mpc': 2.178/1000.*(1.+0.12),\
                 '980Mpc': 3.825/1000.*(1.+0.24), '1600Mpc':5.588/1000.*(1+0.42)} ### Mpc / arcsec (comoving)

    x_FOV = {distance: pixscale[distance]*x_angFOV for distance in ['20Mpc','25Mpc','30Mpc','35Mpc','40Mpc','46Mpc','50Mpc','100Mpc','200Mpc','500Mpc','980Mpc','1600Mpc']}  # cMpc
    y_FOV = {distance: pixscale[distance]*y_angFOV for distance in ['20Mpc','25Mpc','30Mpc','35Mpc','40Mpc','46Mpc','50Mpc','100Mpc','200Mpc','500Mpc','980Mpc','1600Mpc']}  # cMpc

    data = data_tuple[0]; newsize = data_tuple[1]; factor = data_tuple[2];
    
    xystarts = [x_center-x_FOV[distance]/2.,y_center-y_FOV[distance]/2.]
    size     = [x_FOV[distance], y_FOV[distance]]
    
    x1 = ((x_center-x_FOV[distance]/2.)/100.*(newsize/factor))
    x2 = ((x_center+x_FOV[distance]/2.)/100.*(newsize/factor))
    y1 = ((y_center-y_FOV[distance]/2.)/100.*(newsize/factor))
    y2 = ((y_center+y_FOV[distance]/2.)/100.*(newsize/factor))
    
    data_FOV = data[int(round(x1)):int(round(x2)),int(round(y1)):int(round(y2))]
    
    ## Since we cut off the bounding values to the integer, chances are our size is not actually right
    ## Return these values that were calculated from the rounded index values
    xFOV_out = ( int(round(x2)) / (newsize/factor) * 100. ) - ( int(round(x1)) / (newsize/factor) * 100. ) 
    yFOV_out = ( int(round(y2)) / (newsize/factor) * 100. ) - ( int(round(y1)) / (newsize/factor) * 100. )
    size_out = [xFOV_out,yFOV_out]
    xystarts_out = [( int(round(x1)) / (newsize/factor) * 100. ) , ( int(round(y1)) / (newsize/factor) * 100. ) ]
    
    if debug:
        print 'The input FOV is (%s,%s) but due to rounding we are outputting a FOV of size (%s,%s)'\
                % (x_FOV[distance],y_FOV[distance],xFOV_out,yFOV_out)
        print 'The input xystarts that we use (based on the input FOV) is (%s,%s).'% (xystarts[0],xystarts[1])
        print 'The actual xystarts that we use (based on output FOV) is (%s,%s).' % (xystarts_out[0],xystarts_out[1])
    
    return data_FOV, xystarts_out, size_out

In [1]:
### EXAMPLE ###

#print pixscale # in Mpc / arcsec
#pixscale_p =  {'50Mpc': 0.237/1000., '100Mpc': 0.477/1000.,\
#             '200Mpc': 0.928/1000. , '500Mpc': 2.178/1000.} ### Mpc / arcsec (proper)
#print pixscale_p

In [2]:
#0.00044639550000000005 * 3 *60.*60.